In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import spacy

import random
import math
import time

from torchtext.legacy.datasets import TranslationDataset, Multi30k
from torchtext.legacy.data import Field, BucketIterator

Наиболее распространенными моделями последовательностей (seq2seq) являются модели кодер-декодер, которые (обычно) используют рекуррентную нейронную сеть (RNN) для кодирования исходного (входного) предложения в один вектор.  
Вы можете думать о векторе контекста как об абстрактном представлении всего входного предложения. Этот вектор затем декодируется декодером, который учится выводить  предложение, генерируя его по одному слову за раз.

$h_t = \text{Encoder}(x_t, h_{t-1})$

У нас есть последовательность $X = \{x_1, x_2, ..., x_T\}$, где $x_1 = \text{<sos>;}, x_2 = \text{the}$, и так далее. Начальное состояние, $h_0$,  может быть инициализировано вектором из нулей или обучаемым.


Как только последнее слово, $x_T$, был подан на Encoder, мы  используем  информацию в  последнем скрытом состоянии, $h_T$, в зависимости от контекста вектор, т. е. $h_T $ это векторное представление всего исходного предложения.

После получения вектора всего предложения мы можем декодировать предложение уже на новом языке. На каждом шаге декодирования мы подаем правильное слово $y_t$,  дополняем это информацией о скрытом состоянии $s_{t-1}$, где  $s_t = \text{DecoderRNN}(y_t, s_{t-1})$


![alt text](https://i.stack.imgur.com/f6DQb.png)


Мы всегда используем $<sos>$ для первого входа в декодер, $y_1$, но для последующих входов, $y_{\text{from }t; 1}$, мы иногда будем использовать фактическое, основное истинное следующее слово в последовательности, $y_t$, а иногда использовать слово, предсказанное нашим декодером, $\hat{y}_{t-1}$. Использование настоящих токенов в декодере называется Teacher Forcing [можно тут посмотреть](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/)

Мы будем  использовать TorchText и spaCy( как токенизатор) , чтобы помочь вам выполнить всю необходимую предварительную обработку быстрее чем мы делали раньше. В данной работе вам предлагается написать модель Seq2Seq и обучить ее на Multi30k. В данном задание мы будем подавать на вход перевернутые предложения, так как авторы seq2seq считали, что это улучшает качество перевода.

In [ ]:
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
! python -m spacy download en
! python -m spacy download de

spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

     |████████████████████████████████| 12.0 MB 5.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9 MB 5.5 MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-py3-none-any.whl size=14907055 sha256=d2e461d95fce99dc83756d554464f1d4d6a308864fed3abcd4068152c101c7d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-vv70m2ak/wheels/00/66/69/cb6c921610087d2cab339062345098e30a5ceb665360e7b32a
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.loa

In [ ]:
def tokenize_de(text):
    # токенизируем немецкий текст в список токенов и перевернем
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    # токенизиурем английский текст в список токенов
    return [tok.text for tok in spacy_en.tokenizer(text)]

# немецкий язык является полем SRC, а английский в поле TRG
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
# В датасете содержится ~ 30к предложений средняя длина которых 11
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  fields = (SRC, TRG))

downloading training.tar.gz


100%|██████████| 1.21M/1.21M [00:01<00:00, 702kB/s] 


downloading validation.tar.gz


100%|██████████| 46.3k/46.3k [00:00<00:00, 227kB/s]


downloading mmt_task1_test2016.tar.gz


100%|██████████| 66.2k/66.2k [00:00<00:00, 218kB/s]


In [ ]:
labels = ['train', 'validation', 'test']
dataloaders = [train_data, valid_data, test_data]

for d, l in zip(dataloaders, labels):
    print(f"Кол-во предложений {l} : {len(d.examples)}")

Кол-во предложений train : 29000
Кол-во предложений validation : 1014
Кол-во предложений test : 1000


In [ ]:
train_data.examples[1].src

['.',
 'antriebsradsystem',
 'ein',
 'bedienen',
 'schutzhelmen',
 'mit',
 'männer',
 'mehrere']

In [ ]:
train_data.examples[1].trg

['several',
 'men',
 'in',
 'hard',
 'hats',
 'are',
 'operating',
 'a',
 'giant',
 'pulley',
 'system',
 '.']

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print("Кол-во слов исходного", len(SRC.vocab))
print("Кол-во target", len(TRG.vocab))

Кол-во слов исходного 7855
Кол-во target 5893


In [ ]:
SRC.process(["wie geht es dir",])

tensor([[   2],
        [   0],
        [3833],
        [5886],
        [ 664],
        [   0],
        [5886],
        [   0],
        [   0],
        [ 664],
        [5886],
        [7098],
        [ 664],
        [   0],
        [3833],
        [7011],
        [   3]])

## Encoder

Напишем для начала простой Encoder, который реализует следующий функционал:

$ (h_t, c_t) = \text{LSTM}(x_t, (h_{t-1}, c_{t-1}))$

В  методе forward мы передаем исходное предложение $X$, которое преобразуется в embeddings, к которым применяется dropout . Эти вектора затем передаются в RNN. Когда мы передадим всю последовательность RNN, он автоматически выполнит для нас рекуррентный расчет скрытых состояний по всей последовательности.

Вы можете заметить, что мы не передаем начальное скрытое или состояние ячейки в RNN. Это происходит потому, что, как отмечено в документации, если никакое скрытое состояние/ячейки не передается RNN, он автоматически создаст начальное скрытое состояние/ячейки как тензор всех нулей.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        """
        input_dim - это размер / размерность one hot векторов, которые будут вводиться в encoder. Равно размеру входного (исходного) словаря.
        emb_dim - это размерность слоя embedding-a. Этот слой преобразует one-hot векторы в сжатие векторы с размерами emb_dim.
        hid_dim - это размерность скрытого и cell состояний.
        n_layers - это количество слоев в RNN
        dropout - процент dropout
        """
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        # src - предложения размера (src_len x batch_size)
        # embedded = <TODO> 
        embedded = self.embedding(src) # (src_len x batch_size x embd_dim)
        embedded = self.dropout(embedded) # dropout эмбеддингов
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

## Decoder
Похожий на Encoder, но со слоем проекцией, который переводит из hidden_dim в output

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(self.output_dim, self.emb_dim)
        self.rnn = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers) # размер (lstm embd, hid, layers, dropout)
        self.out = nn.Linear(self.hid_dim, self.output_dim) # проекция hid_dim x output_dim
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_, hidden, cell):
        input_ = input_.unsqueeze(0) # 1 размер батча
        # (1 x batch_size x emb_dim)
        # эмбеддинг по input and dropout 
        embedded = self.embedding(input_) 
        embedded = self.dropout(embedded)
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        # batch_size x output_dim
        prediction = self.out(output.squeeze(0)) # проэкция из рнн на выходное

        return prediction, hidden, cell

## Seq2Seq

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self._init_weights()  
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        '''
        src - входное предл (src_len x batch_size)
        trg - выходное предл
        teacher_forcing_ration - 0.5 вероятность получить реальный токен вместо предложенного
        '''
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # сохраняем выходы декодера 
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        # последнее скрытое энкодера используем как первый хидден декодера
        hidden, cell = self.encoder(src) #
        
        # первый инпут декодера - токен <sos> 
        input = trg[0, :]
        
        for t in range(1, max_len):
            output, hidden, cell = self.decoder(input, hidden, cell) # передаем состояние и вход через декодер
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
        
        return outputs
    
    def _init_weights(self):
        p = 0.08
        for name, param in self.named_parameters():
            nn.init.uniform_(param.data, -p, p)

In [ ]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
src_embd_dim =  tgt_embd_dim = 128
hidden_dim = 512
num_layers =  4
dropout_prob = 0.2

batch_size = 256
PAD_IDX = TRG.vocab.stoi['<pad>']

iterators = BucketIterator.splits((train_data, valid_data, test_data),
                                  batch_size = batch_size, device = device)
train_iterator, valid_iterator, test_iterator = iterators

enc = Encoder(input_dim, src_embd_dim, hidden_dim, num_layers, dropout_prob)
dec = Decoder(output_dim, tgt_embd_dim, hidden_dim, num_layers, dropout_prob)
model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 128)
    (rnn): LSTM(128, 512, num_layers=4)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 128)
    (rnn): LSTM(128, 512, num_layers=4)
    (out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [ ]:
test_batch = next(iter(test_iterator))
test_batch


[torchtext.legacy.data.batch.Batch of size 256 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 12x256 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 14x256 (GPU 0)]

In [ ]:
test_batch.src

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   4,    4,    4,  ...,    4,  714,    4],
        [   0,   88,  201,  ...,  669,   12, 1643],
        ...,
        [  16,   32,   13,  ...,    1,    1,    1],
        [   8,    5,    5,  ...,    1,    1,    1],
        [   3,    3,    3,  ...,    1,    1,    1]], device='cuda:0')

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    epoch_loss = 0

    with torch.no_grad():    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) 
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
max_epochs = 20
CLIP = 1

optimizer = optim.Adam(model.parameters(), lr = 1e-4)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

best_valid_loss = float('inf')

for epoch in range(max_epochs):
        
    train_loss = round(train(model, train_iterator, optimizer, criterion, CLIP), 5)
    valid_loss = round(evaluate(model, valid_iterator, criterion),5)
    
    
    if valid_loss < best_valid_loss: # сохраняем модель, если преплексия улучшилась
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/model.pt')
    
    print(f'Эпоха: {epoch} \n Train Loss {train_loss}  Val loss {valid_loss}')
    print(f'Train Perplexity {np.exp(train_loss)}  Val Perplexity {np.exp(valid_loss)}')


Эпоха: 0 
 Train Loss 5.0364  Val loss 4.93117
Train Perplexity 153.9149227374165  Val Perplexity 138.5415111202048
Эпоха: 1 
 Train Loss 4.91984  Val loss 4.80724
Train Perplexity 136.98069452189904  Val Perplexity 122.39334528333556
Эпоха: 2 
 Train Loss 4.86937  Val loss 4.77773
Train Perplexity 130.2388405755698  Val Perplexity 118.83428980455093
Эпоха: 3 
 Train Loss 4.84564  Val loss 4.76274
Train Perplexity 127.18465413051922  Val Perplexity 117.0662483773441
Эпоха: 4 
 Train Loss 4.82351  Val loss 4.74754
Train Perplexity 124.40097277385152  Val Perplexity 115.30029663557252
Эпоха: 5 
 Train Loss 4.80085  Val loss 4.7669
Train Perplexity 121.61374528170383  Val Perplexity 117.55425832751496
Эпоха: 6 
 Train Loss 4.76807  Val loss 4.71777
Train Perplexity 117.69187730115885  Val Perplexity 111.91839615791112
Эпоха: 7 
 Train Loss 4.70703  Val loss 4.73805
Train Perplexity 110.72282429624559  Val Perplexity 114.21127241355805
Эпоха: 8 
 Train Loss 4.65731  Val loss 4.69961
Train 

In [ ]:
test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss} Test PPL: {np.exp(test_loss)}')

Test Loss: 4.603098034858704 Test PPL: 99.79299942936277


In [ ]:
def translate(sentence):
    sent_vec = SRC.process([sentence]).to(device)
    input = torch.zeros((10, 1)).type(torch.LongTensor).to(device)
    input += SRC.vocab.stoi['<sos>']
    output = model(sent_vec, input, 0)
    for t in output:
        if t[0].max(0)[1] != SRC.vocab.stoi['<eos>']:
            print(TRG.vocab.itos[t[0].max(0)[1]], end=' ')
        else:
            break

In [ ]:
translate("wie geht es dir")

gyro a group of a a a a a a 